<a href="https://colab.research.google.com/github/ShivamCholin/CS6910_Assignment_1/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
import random          
import numpy as np      
from time import time    

In [56]:
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))
def dsigmoid(z):
    return sigmoid(z) * (1 - sigmoid(z))

In [57]:
def relu(z):
    return np.maximum(z, 0)
def drelu(z):
    return np.heaviside(z, 1)

In [58]:
def tanh(z):
    return (np.exp(z)-np.exp(-z))/(np.exp(z)+np.exp(-z))
def dtanh(z):
    return 1-tanh(z)**2

In [59]:
def softmax(x):
		e_x = np.exp(x - np.max(x))
		return e_x / np.sum(e_x)

In [60]:
def sqdcost_d(act,y,s):
    act=softmax(act)
    ret=[]
    ll=len(act)
    for j in range(ll):
      add=0
      for i in range(ll):
        add+=2*(act[i]-y[i])*act[i]*((i==j)-act[j])
      ret.append(add)
    return np.array(ret)/s

In [61]:
def cecost_d(act, y,s):
    act=softmax(act)
    act= act-y
    return act

In [62]:
def backward(nn, x, y):
    m = x.shape[1]
    nb = [np.zeros(b.shape) for b in nn.biases]
    nw = [np.zeros(w.shape) for w in nn.weights]
    activation = x 
    acts = [x]
    zs = []    
    for b, w in zip(nn.biases, nn.weights):
        z = np.dot(w, activation) + b 
        zs.append(z)        
        activation = nn.act(z)   
        acts.append(activation)
    delta = nn.cost(acts[-1], y,nn.batch_size) * nn.dact(zs[-1])
    nb[-1] = delta
    nw[-1] = np.dot(delta, acts[-2].transpose()) + nn.lmbda * nn.weights[-1]
    for i in range(2, nn.num_layers):
        z = zs[-i]
        sp = nn.dact(z)
        delta = np.dot(nn.weights[-i + 1].transpose(), delta) * sp
        nb[-i] = delta
        nw[-i] = np.dot(delta, acts[-i - 1].transpose())+ nn.lmbda * nn.weights[-i]
    return (nb, nw)

In [63]:
def nagbackward(nn, x, y):
    nb = [np.zeros(b.shape) for b in nn.biases]
    nw = [np.zeros(w.shape) for w in nn.weights]
    activation = x 
    acts = [x]
    zs = []  
    new_weights = [w-v*nn.gamma for w,v in zip(nn.weights,nn.vw)]
    new_biases = [b-v*nn.gamma for b,v in zip(nn.biases,nn.vb)]
    for b, w in zip(new_biases, new_weights):
        z = np.dot(w, activation) + b 
        zs.append(z)        
        activation = nn.act(z)   
        acts.append(activation)
    delta = nn.cost(acts[-1], y,nn.batch_size) * nn.dact(zs[-1])
    nb[-1] = delta
    nw[-1] = np.dot(delta, acts[-2].transpose())+ nn.lmbda * nn.weights[-1]
    for i in range(2, nn.num_layers):
        z = zs[-i]
        sp = nn.dact(z)
        delta = np.dot(new_weights[-i + 1].transpose(), delta) * sp
        nb[-i] = delta
        nw[-i] = np.dot(delta, acts[-i - 1].transpose())+ nn.lmbda * nn.weights[-i]
    return (nb, nw)

In [64]:
def learn(nn, tr_data):
    n = len(tr_data)
    for j in range(nn.epochs):
        time_start=time()
        random.shuffle(tr_data)
        batches = [tr_data[k: k + nn.batch_size] for k in range(0,n,nn.batch_size)]
        for batch in batches:
            nn.grad(nn, batch)
        time_end=time()
        print('Epoch {0}:time taken {1} seconds, accuracy {2}%'.format(f'{j + 1:2}',1.0*time_end-time_start, 100.0 * evaluate2(nn, tr_data) / len(tr_data)))

In [65]:
def forward(nn, a):
    for b, w in zip(nn.biases, nn.weights):
        a = nn.act(np.dot(w, a) + b)
    return a

In [66]:
def sgd(nn, batch):
    nb = [np.zeros(b.shape) for b in nn.biases]
    nw = [np.zeros(w.shape) for w in nn.weights]
    num=1
    for x, y in batch:
        dnb, dnw = backward(nn, x, y)
        nb = [nb + dnb for nb, dnb in zip(nb, dnb)]
        nw = [nw + dnw for nw, dnw in zip(nw, dnw)]
    nn.weights = [w - (nn.learning_rate ) * nw for w, nw in zip(nn.weights, nw)]
    nn.biases  = [b - (nn.learning_rate) * nb for b, nb in zip(nn.biases, nb)]

In [67]:
def mbgd(nn, batch):
    nb = [np.zeros(b.shape) for b in nn.biases]
    nw = [np.zeros(w.shape) for w in nn.weights]
    for x, y in batch:
        dnb, dnw = backward(nn, x, y) 

        nb = [nb + dnb for nb, dnb in zip(nb, dnb)]
        nw = [nw + dnw for nw, dnw in zip(nw, dnw)]

    nn.vw = [vweight*nn.gamma + nn.learning_rate * nw for vweight,nw in zip(nn.vw,nw)]
    nn.vb = [vbiases*nn.gamma + nn.learning_rate * nb for vbiases,nb in zip(nn.vb,nb)]
    nn.weights = [w - v for w, v in zip(nn.weights, nn.vw)]
    nn.biases  = [b - v for b, v in zip(nn.biases, nn.vb)]

In [68]:
def nag(nn, batch):
    nb = [np.zeros(b.shape) for b in nn.biases]
    nw = [np.zeros(w.shape) for w in nn.weights]
    for x, y in batch:
        dnb, dnw = nagbackward(nn, x, y) 

        nb = [nb + dnb for nb, dnb in zip(nb, dnb)]
        nw = [nw + dnw for nw, dnw in zip(nw, dnw)]

    nn.vw = [vweight*nn.gamma + nn.learning_rate * nw for vweight,nw in zip(nn.vw,nw)]
    nn.vb = [vbiases*nn.gamma + nn.learning_rate * nb for vbiases,nb in zip(nn.vb,nb)]
    nn.weights = [w - v for w, v in zip(nn.weights, nn.vw)]
    nn.biases  = [b - v for b, v in zip(nn.biases, nn.vb)]

In [69]:
def rmsprop(nn, batch):
    nb = [np.zeros(b.shape) for b in nn.biases]
    nw = [np.zeros(w.shape) for w in nn.weights]
    for x, y in batch:
        dnb, dnw = backward(nn, x, y) 

        nb = [nb + dnb for nb, dnb in zip(nb, dnb)]
        nw = [nw + dnw for nw, dnw in zip(nw, dnw)]

    nn.gew = [gew*nn.gamma + (1.0-nn.gamma)*np.square(nw) for gew,nw in zip(nn.gew,nw)]
    nn.geb = [geb*nn.gamma + (1.0-nn.gamma)*np.square(nb) for geb,nb in zip(nn.geb,nb)]
    nn.weights = [w - nn.learning_rate * nw / np.sqrt(ge+nn.epsilon) for w, ge,nw in zip(nn.weights, nn.gew,nw)]
    nn.biases  = [b - nn.learning_rate * nb / np.sqrt(ge+nn.epsilon) for b, ge,nb in zip(nn.biases, nn.geb,nb)]

In [70]:
def adam(nn, batch):
    nb = [np.zeros(b.shape) for b in nn.biases]
    nw = [np.zeros(w.shape) for w in nn.weights]
    for x, y in batch:
        dnb, dnw = backward(nn, x, y) 
        nb = [nb + dnb for nb, dnb in zip(nb, dnb)]
        nw = [nw + dnw for nw, dnw in zip(nw, dnw)]

    nn.mw = [nn.beta1w*m + (1.0-nn.beta1w)*nw for m,nw in zip(nn.mw,nw)]
    nn.mb = [nn.beta1b*m + (1.0-nn.beta1b)*nb for m,nb in zip(nn.mb,nb)]
    nn.vw = [nn.beta2w*v + (1.0-nn.beta2w)*np.square(nw) for v,nw in zip(nn.vw,nw)]
    nn.vb = [nn.beta2b*v + (1.0-nn.beta2b)*np.square(nb) for v,nb in zip(nn.vb,nb)]
    nn.beta1_expw *= nn.beta1w
    nn.beta2_expw *= nn.beta2w
    nn.beta1_expb *= nn.beta1b
    nn.beta2_expb *= nn.beta2b
    nn.weights = [w - nn.learning_rate *(m/(1.0-nn.beta1_expw))/ (np.sqrt(v / (1.0 - nn.beta2_expw)) + nn.epsilon) for w, m,v in zip(nn.weights, nn.mw,nn.vw)]
    nn.biases = [b - nn.learning_rate *(m/(1.0-nn.beta1_expb))/ (np.sqrt(v / (1.0 - nn.beta2_expb)) + nn.epsilon) for b, m,v in zip(nn.biases, nn.mb,nn.vb)]

In [71]:
def nadam(nn, batch):
    nb = [np.zeros(b.shape) for b in nn.biases]
    nw = [np.zeros(w.shape) for w in nn.weights]
    for x, y in batch:
        dnb, dnw = backward(nn, x, y) 

        nb = [nb + dnb for nb, dnb in zip(nb, dnb)]
        nw = [nw + dnw for nw, dnw in zip(nw, dnw)]


    nn.mw = [nn.beta1w*m + (1.0-nn.beta1w)*nw for m,nw in zip(nn.mw,nw)]
    nn.mb = [nn.beta1b*m + (1.0-nn.beta1b)*nb for m,nb in zip(nn.mb,nb)]
    nn.vw = [nn.beta2w*v + (1.0-nn.beta2w)*np.square(nw) for v,nw in zip(nn.vw,nw)]
    nn.vb = [nn.beta2b*v + (1.0-nn.beta2b)*np.square(nb) for v,nb in zip(nn.vb,nb)]
    #nn.weights = [w - (eta ) * nw for w, nw in zip(nn.weights, nw)]
    #nn.biases  = [b - (eta ) * nb for b, nb in zip(nn.biases, nb)]
    nn.beta1_expw *= nn.beta1w
    nn.beta2_expw *= nn.beta2w
    nn.beta1_expb *= nn.beta1b
    nn.beta2_expb *= nn.beta2b
    mwn=[m/(1-nn.beta1_expw) for m in nn.mw]
    vwn=[v/(1-nn.beta2_expw) for v in nn.vw]
    mbn=[m/(1-nn.beta1_expb) for m in nn.mb]
    vbn=[v/(1-nn.beta2_expb) for v in nn.vb]
    mwnn=[nn.beta1w * m + ((1-nn.beta1w)/(1-nn.beta1_expw))*g for m,g in zip(mwn,nw)]
    mbnn=[nn.beta1b * m + ((1-nn.beta1b)/(1-nn.beta1_expb))*g for m,g in zip(mbn,nb)]

    nn.weights = [w - nn.learning_rate *m/ (np.sqrt(v) + nn.epsilon) for w, m,v in zip(nn.weights, mwnn,vwn)]
    nn.biases = [b - nn.learning_rate *m/ (np.sqrt(v)+ nn.epsilon) for b, m,v in zip(nn.biases, mbnn,vbn)]

In [72]:
class Network:
    num_layers=0
    biases=[]
    weights=[]
    def __init__(self,nl,x,y,act1,act2,lmbda,g_func,c_func,lr,bs,e):
      self.num_layers=nl
      self.biases=x
      self.weights=y
      self.act=act1
      self.dact=act2
      self.grad=g_func
      self.vw=[np.zeros(w.shape) for w in self.weights]
      self.vb=[np.zeros(b.shape) for b in self.biases]
      self.mw=[np.zeros(w.shape) for w in self.weights]
      self.mb=[np.zeros(b.shape) for b in self.biases]
      self.gew=[np.zeros(w.shape) for w in self.weights]
      self.geb=[np.zeros(b.shape) for b in self.biases]
      self.gamma=0.9
      self.epsilon = 1e-8
      self.beta1w = 0.9
      self.beta2w = 0.999
      self.beta1_expw = 1.0
      self.beta2_expw = 1.0
      self.beta1b = 0.9
      self.beta2b = 0.999
      self.beta1_expb = 1.0
      self.beta2_expb = 1.0
      self.lmbda=lmbda
      self.cost=c_func
      self.learning_rate=lr
      self.epochs=e
      self.batch_size=bs

def init_network(layers,actfunc,lmbda,grad_function,Loss_f,lr,bs,e):
    if actfunc=="tanh":
      act1=tanh
      act2=dtanh
    elif actfunc=="sigmoid":
      act1=sigmoid
      act2=dsigmoid
    else:
      act1=relu
      act2=drelu
    if grad_function=="SGD":
      g_func=sgd
    elif grad_function=="MBGD":
      g_func=mbgd
    elif grad_function=="NAG":
      g_func=nag
    elif grad_function=="RMSPROP":
      g_func=rmsprop
    elif grad_function=="ADAM":
      g_func=adam
    elif grad_function=="NADAM":
      g_func=nadam
    if Loss_f=="MSE":
      cost_func=sqdcost_d
    else:
      cost_func=cecost_d
    return Network(len(layers),[np.random.randn(y, 1) for y in layers[1:]],[np.random.randn(y, x) for x, y in zip(layers[:-1], layers[1:])],act1,act2,lmbda,g_func,cost_func,lr,bs,e)

In [73]:
def evaluate(nn, te_data):
    test_results = [(np.argmax(forward(nn, x)), y) for (x, y) in te_data]
    return sum(int(x == y) for (x, y) in test_results)

In [74]:
def evaluate2(nn, te_data):
    test_results = [(np.argmax(forward(nn, x)), y) for (x, y) in te_data]
    return sum(int(x == np.argmax(y)) for (x, y) in test_results)

In [75]:
import keras
def change_y(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e
fashion_mnist = keras.datasets.fashion_mnist
tr_data, te_data = fashion_mnist.load_data()
training_x = [np.reshape(x, (784, 1))/255 for x in tr_data[0]]
training_y = [change_y(y) for y in tr_data[1]]
tr_data = zip(training_x, training_y)
test_inputs = [np.reshape(x, (784, 1))/255 for x in te_data[0]]
te_data = zip(test_inputs, te_data[1])
tr_data=list(tr_data)
te_data=list(te_data)

In [77]:
no_of_hlayers=2
size_of_each_hlayer=32
# Activation functions can be "sigmoid" , "tanh" and "Relu"
Activation_function="sigmoid"
L2_regularisation=0
#optimization functions can be "SGD" , "MBGD" , "NAG" , "RMSPROP" , "ADAM", "NADAM"
Gradient_descent_optimizer="SGD"
epochs = 10
batch_size =10
learning_rate = 0.3
# loss to optimize , can be "Cross_entropy" or "MSE"
loss_function = "Cross_entropy"
L=[]
L.append(784)
for _ in range(no_of_hlayers):
  L.append(size_of_each_hlayer)
L.append(10)
nn = init_network(L,Activation_function,L2_regularisation,Gradient_descent_optimizer,loss_function,learning_rate,batch_size,epochs)
print('start')
learn(nn, tr_data)
print('accuracy {0}%'.format(100.0 * evaluate(nn, te_data) / len(te_data)))

start
Epoch  1:time taken 21.90295958518982 seconds, accuracy 72.11166666666666%
Epoch  2:time taken 21.917940139770508 seconds, accuracy 72.59833333333333%
Epoch  3:time taken 22.307148933410645 seconds, accuracy 76.59%
Epoch  4:time taken 21.885160207748413 seconds, accuracy 76.47833333333334%
Epoch  5:time taken 22.096740245819092 seconds, accuracy 78.14666666666666%
Epoch  6:time taken 21.90775442123413 seconds, accuracy 75.72666666666667%
Epoch  7:time taken 21.84419083595276 seconds, accuracy 75.34333333333333%
Epoch  8:time taken 21.879703044891357 seconds, accuracy 73.76333333333334%
Epoch  9:time taken 21.81201958656311 seconds, accuracy 78.73333333333333%
Epoch 10:time taken 21.980120420455933 seconds, accuracy 78.78333333333333%
accuracy 77.16%
